# 04 - Bayesian Assurance Model (BAM) for Performance Metrics

**Purpose**: This notebook implements the Bayesian Assurance Model (BAM) workflow for planning new studies to determine the optimal sample size for achieving a desired precision (Highest Posterior Density Interval width) for key diagnostic metrics like Sensitivity and Specificity.

**Inputs**:
- Pilot study data, represented by `PILOT_RATE`, `TOTAL_N`, and `CORRECT_N`, which define the prior for the Bayesian model.

**Outputs**:
- `bac_sensitivity_{width}.png`, `bac_specificity_{width}.png`: Assurance curve plots showing the probability of achieving the target HPD width at different sample sizes.
- `bammer_accuracy.pkl`: A pickled `Bammer` object containing the full state of the last-run analysis (for Specificity).

### 4.1 BAM Implementation and Execution

This cell contains the complete implementation and execution of the Bayesian Assurance Method simulation.

- **Imports and Setup**: Imports necessary libraries (`pymc`, `arviz`, `numpy`, `matplotlib`) and sets up global constants for the simulation (`NUM_CORES`, `MAX_SAMPLE`, `PILOT_RATE`, etc.).
- **`BamResult` Class**: A dataclass to store and process the results of a BAM simulation for a single target width, including methods to interpolate between sample sizes and assurances.
- **`Metric` Enum**: Defines the diagnostic metrics that can be analyzed (e.g., Sensitivity, Specificity).
- **`Bammer` Class**: The main orchestration class. It takes a metric and pilot data, then runs the simulation across a range of sample sizes to calculate the probability (assurance) of achieving a target HPD width.
- **Simulation Loop**: The `_plot_assurance_curve` method contains the core simulation logic:
    1. It iterates through different sample sizes (`n`).
    2. For each `n`, it simulates 500 hypothetical future studies using `pm.sample_prior_predictive`.
    3. For each simulated study, it updates the Bayesian model and calculates the resulting HPD width.
    4. It computes "assurance" as the proportion of simulated studies that met the target HPD width.
- **Execution**: The final loop instantiates the `Bammer` for Sensitivity and Specificity and runs the analysis for a target width of 0.2.

### 4.2 Persist BAM Analysis State

This cell uses `pickle` to save the state of the last-run `Bammer` object (`bam`) to a file. This allows the results of the time-consuming simulation to be saved and reloaded later for further analysis or visualization without needing to re-run the entire process.

In [ ]:
# %reload_ext autoreload
# %autoreload 2
import pickle
import logging

import numpy as np

from early_markers.cribsy.common.constants import RAND_STATE, PKL_DIR
from early_markers.cribsy.common.enums import Metric
from early_markers.cribsy.common.bayesian_assurance import Bammer


pymc_logger = logging.getLogger("pymc")
pymc_logger.setLevel(logging.ERROR)  # or logging.WARNING to allow warnings

np.random.seed(RAND_STATE)

NUM_CORES = 12
MAX_SAMPLE = 500
PILOT_RATE = 0.796
TOTAL_N = 54
CORRECT_N = round(PILOT_RATE * TOTAL_N)

TARGET_WIDTH = 0.1


for metric in [Metric.SENS, Metric.SPEC]:
    bam = Bammer(
        metric=metric,
        numerator=CORRECT_N,
        denominator=TOTAL_N,
        max_sample=MAX_SAMPLE,
        num_cores=NUM_CORES,
    )
    bam.set_bams_for_widths([0.2])

with open(PKL_DIR / f"bammer_accuracy.pkl", "wb") as f:
    pickle.dump(bam, f)